# Creating Dates Dataset 

this notebook is used for collecting and classifiying dates dataset

## Collecting Datasets

Using Selenium webDriver for fetching images from googlethe documentaion can be found [here](https://www.selenium.dev/selenium/docs/api/py/index.html)

In [ ]:
# install Selenium
!pip install -U selenium

In [5]:
import os 
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 
from urllib import request
from urllib.request import urlopen
import json
#Opens up web driver and goes to Google Images
driver = webdriver.Chrome("C:\\My Projects\\webDrivers\\chromedriver.exe")
header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}


C:\Users\Saleh\AppData\Local\Temp/ipykernel_14712/280538312.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\My Projects\\webDrivers\\chromedriver.exe")


In [3]:
keywords = {'Ajwa':'Ajwa date',
           'Sokari':'Sokari dates',
            'Medjool':'medjool dates , تمر مجدول',
            'Nabtat Ali':'nabtat ali dates , تمر نبته علي',
            'Shaishe':'تمر شيشي',
            'Sugaey':'تمر صقعي',
           }

In [5]:
# Create the directories

parent_dir = "{0}\\dates_dataset\\dataset_collected_google_images".format(os.getcwd())
for keyword in keywords:
    path = os.path.join(parent_dir, keyword)
    if(os.path.isdir(path)):
        pass
    else:
        os.mkdir(path)
        print("Directory '% s' created" % keyword)

In [79]:
def search_google_images (keyword,limit,query):
    driver.get('https://www.google.com.sa/imghp?hl=ar&ogbl')
    # Search box 
    inputBox = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
    inputBox.send_keys(query)
    inputBox.send_keys(Keys.ENTER)

    #Will keep scrolling down the webpage until it cannot scroll no more
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        time.sleep(2)
        new_height = driver.execute_script('return document.body.scrollHeight')
        try:
            driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()
            time.sleep(2)
        except:
            pass
        if new_height == last_height:
            break
        last_height = new_height

    """
    Finding xpath patterns for images 
    first image = //*[@id="islrg"]/div[1]/div[3]/a[1]/div[1]/img
    second image = //*[@id="islrg"]/div[1]/div[2]/a[1]/div[1]/img

    second div is chainging 

    """
    path = os.path.join(parent_dir, keyword)
    successCounter = 0
    images = []
    for i in range(limit):
        try:
            img = driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div['+str(i)+']/a[1]/div[1]/img')
            images.append(img)
        except:
            pass

    print('Number of scraped images:', len(images))
    
    for image in images:
        try:
            image.screenshot(os.path.join(path ,str(successCounter) + "_2nd_." + 'jpg'))
            successCounter  += 1
        except:
            print ("can't get img")
    print('Number of saved images : ',successCounter)
    

In [ ]:
for key,val in keywords.items():
    print('Fetching {} images ..'.format(key))
    search_google_images(key,400,val)

### Clean dataset
Dataset after cleaning

In [8]:
for _dir in os.listdir(parent_dir):
    print('Number of {} :'.format(_dir),len([name for name in os.listdir(os.path.join(parent_dir,_dir)) if os.path.isfile(os.path.join(os.path.join(parent_dir,_dir), name))]))


Number of Ajwa : 530
Number of Medjool : 284
Number of Nabtat Ali : 27
Number of Shaishe : 209
Number of Sokari : 325
Number of Sugaey : 333


## Combine datasets

In [ ]:
# Create Combined Directory Dataset
import shutil


controlled_dir = '{0}\\dates_dataset\\dates_controlled_env'.format(os.getcwd())
combined_dir = '{0}\\dates_dataset\\dataset_combined'.format(os.getcwd())
if(not os.path.isdir(combined_dir)):
    os.mkdir(combined_dir)
def copyFilesToDirectory(source_dir,dist_dir):
    for file_name in os.listdir(source_dir):
        # construct full file path
        source = source_dir +'\\'+ file_name
        destination = dist_dir +'\\'+  file_name
        # copy only files
        if os.path.isfile(source):
            if not os.path.exists(destination):
                shutil.copy(source, destination)
                print('copied', file_name)
            else:
                base, extension = os.path.splitext(destination)
                i = 1
                while os.path.exists(os.path.join(dist_dir, '{}_{}{}'.format(base, i, extension))):
                    i += 1
                shutil.copy(file_path, os.path.join(out_dir, '{}_{}{}'.format(base, i, extension)))
            
            
for key in keywords:
    combined_key_dir = os.path.join(combined_dir,key)
    if (not os.path.isdir(combined_key_dir)):
        os.mkdir(key_dir)
    key_google_dir = os.path.join(parent_dir,key)
    key_controlled_env_dir = os.path.join(controlled_dir,key)
    copyFilesToDirectory(key_google_dir,combined_key_dir)
    copyFilesToDirectory(key_controlled_env_dir,combined_key_dir)
    

In [ ]:
# Rename Files 

for key in keywords:
    path = os.path.join(combined_dir,key)
    files = os.listdir(path)
    for index, file in enumerate(files):
        os.rename(os.path.join(path, file), os.path.join(path, ''.join([str(index), '.jpg'])))
    print('All Files in {} directory has been renamed.'.format(key))

### Final Dataset Size 

In [7]:
for key in keywords:
    print('{0} Type has {1} of images'.format(key,len(os.listdir(os.path.join(combined_dir,key)))))

Ajwa Type has 517 of images
Sokari Type has 589 of images
Medjool Type has 419 of images
Nabtat Ali Type has 204 of images
Shaishe Type has 380 of images
Sugaey Type has 501 of images
